# Word2vec

In [1]:
# 親のフォルダのパスを追加
import sys; sys.path.insert(0, '..')

from util.data_loader import DataLoader
from util.metric_calculator import MetricCalculator

In [2]:
# Movielensのデータの読み込み
data_loader = DataLoader(num_users=1000, num_test_items=5, data_path='../data/ml-10M100K/')
movielens = data_loader.load()

In [3]:
import gensim
import logging

movie_content = movielens.item_content.copy()

# tagが付与されていない映画もあるが、genreはすべての映画に付与されている
# tagとgenreを結合したものを映画のコンテンツ情報として似ている映画を探して推薦していく
# tagがない映画に関しては、NaNになっているので、空のリストに変換してから処理をする
movie_content['tag_genre'] = movie_content['tag'].fillna("").apply(list) + movie_content['genre'].apply(list)
movie_content['tag_genre'] = movie_content['tag_genre'].apply(lambda x:set(map(str, x)))

# タグとジャンルデータを使って、word2vecを学習する
tag_genre_data = movie_content.tag_genre.tolist()
model = gensim.models.word2vec.Word2Vec(tag_genre_data, vector_size=100, window=100, sg=1, hs=0, epochs=50, min_count=5)


/Users/mk/.pyenv/versions/3.7.8/envs/recommender-systems-at-work3.7.8/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [4]:
# animeタグに似ているタグを確認
model.wv.most_similar('anime')

[('studio ghibli', 0.8409439921379089),
 ('zibri studio', 0.8167261481285095),
 ('pelicula anime', 0.797528862953186),
 ('miyazaki', 0.786034107208252),
 ('hayao miyazaki', 0.7857083678245544),
 ('japan', 0.6094456911087036),
 ('Animation', 0.5489083528518677),
 ('wilderness', 0.5156647562980652),
 ('steampunk', 0.49222904443740845),
 ('environmental', 0.4773883521556854)]

In [5]:
# Word2vecContentレコメンド
from src.word2vec import Word2vecRecommender
recommender = Word2vecRecommender()
recommend_result = recommender.recommend(movielens)

In [6]:
#  評価
metric_calculator = MetricCalculator()
metrics = metric_calculator.calc(
    movielens.test.rating.tolist(), recommend_result.rating.tolist(),
    movielens.test_user2items, recommend_result.user2items, k=10)
print(metrics)

rmse=0.000, Precision@K=0.010, Recall@K=0.033
